In [1]:
import cv2
import numpy as np

In [2]:
cap = cv2.VideoCapture(0)
_, img = cap.read()
cv2.imshow('image', img)
cv2.waitKey(0)
cap.release()
cv2.destroyAllWindows()

In [3]:
def save_initial():
    '''Used to save initial picture of the speedometer to be later used for different computations. 
    Saves a "reference.jpg" file in the current working directory'''
    try:
        cap = cv2.VideoCapture(0)
        while True:
            _,frame = cap.read()
            frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            frame_gray_copy = np.copy(frame_gray)
            cv2.putText(frame_gray_copy, "Press 's' to save image", (0,25), 2, 0.9, (255,255,255), 1, cv2.LINE_AA, False)
            cv2.line(frame_gray_copy, (0,30), (frame_gray_copy.shape[1], 30), (255,255,255))
            cv2.imshow('frame', frame_gray_copy)
            k = cv2.waitKey(1) & 0xFF
            if k in [115, 83]:
                cv2.imwrite('reference.jpg', frame_gray)
                print('Image Saved')
                break
            elif k == 27:
                break
    except:
        print('An error has occurred.')
    cap.release()
    cv2.destroyAllWindows()

In [4]:
def compute_sum_thresh(frame, tolerance_percent=5):
    return_value = {'pixel_sum':-1, 'lower_limit':-1, 'upper_limit':-1}
    return_value['pixel_sum'] = np.sum(frame/ frame.size)
    return_value['lower_limit'] = return_value['pixel_sum']*(1-tolerance_percent/100)
    return_value['upper_limit'] = return_value['pixel_sum']*(1+tolerance_percent/100)
    return return_value

In [5]:
def compare(filename='reference.jpg'):
    tolerance_percent = 3
    # Read the image
    reference_img = cv2.imread('reference.jpg',0)
    cv2.imshow('Reference image', reference_img)
    ref_img_values = compute_sum_thresh(reference_img, tolerance_percent)
    print("\nDisplaying reference image.\nPress 'Esc' to exit, any other key to continue.")
    k = cv2.waitKey(0)
    if k == 27:
        cv2.destroyAllWindows()
        return
    
    cap = cv2.VideoCapture(0)
    while True:
        _,frame = cap.read()
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        frame_values = compute_sum_thresh(frame_gray, tolerance_percent)
        
        if frame_values['pixel_sum'] > ref_img_values['upper_limit'] or frame_values['pixel_sum'] < ref_img_values['lower_limit']:
            cv2.putText(frame, 'NG', (10,25), 2, 1.2, (0,0,255), 1, cv2.LINE_AA)
        else:
            cv2.putText(frame, 'OK', (10,25), 2, 1.2, (0,255,0), 1, cv2.LINE_AA)
        cv2.imshow('Output', frame)
        
        k = cv2.waitKey(1) & 0xFF
        if k == 27:
            break
    cv2.destroyAllWindows()

In [6]:
save_initial()

Image Saved


In [7]:
compare()


Displaying reference image.
Press 'Esc' to exit, any other key to continue.
